# WOfS Validation Data Analysis - Water Body Type Classification  <img align="right" src="../Supplementary_data/DE_Africa_Logo_Stacked_RGB_small.jpg">

* **Products used:** 
[ga_ls8c_wofs_2](https://explorer.digitalearth.africa/ga_ls8c_wofs_2),
[ga_ls8c_wofs_2_annual_summary ](https://explorer.digitalearth.africa/ga_ls8c_wofs_2_annual_summary)

## Background
The [Water Observations from Space (WOfS)](https://www.ga.gov.au/scientific-topics/community-safety/flood/wofs/about-wofs) is a derived product from Landsat 8 satellite observations as part of provisional Landsat 8 Collection 2 surface reflectance and shows surface water detected in Africa.
Individual water classified images are called Water Observation Feature Layers (WOFLs), and are created in a 1-to-1 relationship with the input satellite data. 
Hence there is one WOFL for each satellite dataset processed for the occurrence of water.

The data in a WOFL is stored as a bit field. This is a binary number, where each digit of the number is independantly set or not based on the presence (1) or absence (0) of a particular attribute (water, cloud, cloud shadow etc). In this way, the single decimal value associated to each pixel can provide information on a variety of features of that pixel. 
For more information on the structure of WOFLs and how to interact with them, see [Water Observations from Space](../Datasets/Water_Observations_from_Space.ipynb) and [Applying WOfS bitmasking](../Frequently_used_code/Applying_WOfS_bitmasking.ipynb) notebooks.

## Description
This notebook explains how you can perform validation analysis for WOFS derived product using collected ground truth dataset and window-based sampling. 

The notebook demonstrates how to:

1. Load validation points for each partner institutions following cleaning stage as an ESRI shapefile
2. Query WOFL data for validation points and capture available WOfS observation available
3. Extract statistics for each WOfS observation in each validation point using a 3 by 3 window and multiprocessing functionality 
4. Extract a LUT for each point that contains both validation info and WOfS result for each month 
***

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

After finishing the analysis, you can modify some values in the "Analysis parameters" cell and re-run the analysis to load WOFLs for a different location or time period.

### Load packages
Import Python packages that are used for the analysis.

In [19]:
%matplotlib inline

import time 
import datacube
from datacube.utils import masking, geometry 
import sys
import os
import dask 
import rasterio, rasterio.features
import xarray
import glob
import numpy as np
import pandas as pd
import seaborn as sn
import geopandas as gpd
import subprocess as sp
import matplotlib.pyplot as plt
import scipy, scipy.ndimage
import warnings
warnings.filterwarnings("ignore") #this will suppress the warnings for multiple UTM zones in your AOI 

sys.path.append("../Scripts")
from rasterio.mask import mask
from geopandas import GeoSeries, GeoDataFrame
from shapely.geometry import Point
from deafrica_plotting import map_shapefile,display_map, rgb
from deafrica_spatialtools import xr_rasterize
from deafrica_datahandling import wofs_fuser, mostcommon_crs,load_ard,deepcopy
from deafrica_dask import create_local_dask_cluster

#for parallelisation 
from multiprocessing import Pool, Manager
import multiprocessing as mp
from tqdm import tqdm

### Analysis parameters

To analyse validation points collected by each partner institution, we need to obtain WOfS surface water observation data that corresponds with the labelled input data locations. 

### Loading Dataset

1. Load validation points for each partner institutions as a list of observations each has a location and month
    * Load the cleaned validation file as ESRI `shapefile`
    * Inspect the shapefile

In [20]:
#Read the ground truth data following analysis step 
CEO = '../Supplementary_data/Validation/Refined/Continent/AfriGIST_inland_5D.csv'
input_data = pd.read_csv(CEO,delimiter=",")

In [21]:
input_data['WET_T'] = input_data['CLASS_WET'].groupby(input_data['PLOT_ID']).transform('sum')

In [22]:
input_data['CLEAR_T'] = input_data['CLEAR_OBS'].groupby(input_data['PLOT_ID']).transform('sum')

In [23]:
Summarize = input_data.groupby('PLOT_ID',as_index=False,sort=False).last()

In [24]:
Summarize['FREQUENCY'] = Summarize['WET_T']/Summarize['CLEAR_T']

In [25]:
Summarize.loc[Summarize['WET_T'] == 0.0, 'TYPE'] = 'Dry'
Summarize.loc[(Summarize['WET_T'] != 0.0) & (Summarize['FREQUENCY'] <= 0.9), 'TYPE'] = 'Ephemeral'  
Summarize.loc[(Summarize['WET_T'] != 0.0) & (Summarize['FREQUENCY'] > 0.9), 'TYPE'] = 'Permanent'  

In [26]:
Summarize 

,PLOT_ID,Unnamed: 0,LON,LAT,CLASS,MONTH,ACTUAL,CLASS_WET,CLEAR_OBS,PREDICTION,geometry,WET_T,CLEAR_T,FREQUENCY,TYPE
0,137483175.0,6,30.463813,-26.653807,Open water - freshwater,12,1,1.0,1.0,1,POINT (30.46381301 -26.65380659),7.0,12.0,0.583333,Ephemeral
1,137483176.0,14,30.026031,-26.673227,Open water - Constructed (e.g. aquaculture),12,1,0.0,2.0,0,POINT (30.02603057 -26.67322664),2.0,16.0,0.125000,Ephemeral
2,137483177.0,19,31.700362,-26.746737,Open water - freshwater,10,1,1.0,2.0,1,POINT (31.70036188 -26.74673726),5.0,10.0,0.500000,Ephemeral
3,137483178.0,24,31.937287,-26.801901,Open water - freshwater,10,1,0.0,2.0,0,POINT (31.93728675 -26.80190076),4.0,9.0,0.444444,Ephemeral
4,137483179.0,32,27.339949,-26.863925,Open water - freshwater,12,1,1.0,3.0,1,POINT (27.33994919 -26.86392537),8.0,19.0,0.421053,Ephemeral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,137482799.0,2758,13.254423,-8.829394,Urban (Settlements/ roads),9,0,0.0,1.0,0,POINT (13.25442329 -8.82939446),0.0,3.0,0.000000,Dry
533,137482800.0,2762,13.848290,-8.953839,Open water - freshwater,9,0,1.0,1.0,1,POINT (13.8482901 -8.953838786),3.0,4.0,0.750000,Ephemeral
534,137482802.0,2766,13.607012,-9.155465,Open water - freshwater,9,0,1.0,1.0,1,POINT (13.60701228 -9.155465),2.0,2.0,1.000000,Permanent
535,137482803.0,2772,20.258942,-9.360062,Open water - freshwater,12,1,1.0,2.0,1,POINT (20.25894246 -9.360061605),6.0,12.0,0.500000,Ephemeral


In [27]:
Summarize.to_csv(('../Supplementary_data/Validation/Refined/Continent/WOfS_Summary/AfriGIST_inland_5D_Summarize_T0.9.csv'))

In [ ]:
print(datacube.__version__)

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Last modified:** September 2020

**Compatible datacube version:** 

## Tags
Browse all available tags on the DE Africa User Guide's [Tags Index](https://) (placeholder as this does not exist yet)